# Exploratory Data Analysis(EDA) 

In [1]:
# import the libraries
import pandas as pd
import numpy as np

In [ ]:
# import utility modules via Url
import httpimport

address = ""

with httpimport.remote_repo(["utility"], address):
    from utility import *